In [1]:
import psutil
import csv
from datetime import datetime
import time

In [2]:
! pip install psycopg2-binary
! pip install pyodbc



   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   --------- ------------------------------ 0.3/1.2 MB ? eta -:--:--
   ------------------ --------------------- 0.5/1.2 MB 1.0 MB/s eta 0:00:01
   --------------------------- ------------ 0.8/1.2 MB 1.1 MB/s eta 0:00:01
   ------------------------------------ --- 1.0/1.2 MB 1.1 MB/s eta 0:00:01
   ------------------------------------ --- 1.0/1.2 MB 1.1 MB/s eta 0:00:01
   ------------------------------------ --- 1.0/1.2 MB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 671.0 kB/s eta 0:00:00


PostgreSQL

In [ ]:
-- PostgreSQL version
CREATE TABLE process_metrics (
    timestamp TEXT,
    name TEXT,
    pid INTEGER,
    ws BIGINT,
    vm BIGINT,
    private_memory BIGINT,
    working_set BIGINT,
    virtual_memory BIGINT,
    paged_memory BIGINT,
    peak_working_set BIGINT,
    peak_virtual_memory BIGINT,
    cpu_user FLOAT,
    cpu_system FLOAT,
    cpu_total FLOAT,
    processor_affinity TEXT,
    start_time TEXT,
    has_exited BOOLEAN,
    responding BOOLEAN
);



In [ ]:
#python script for PostgreSQL
import psutil
import psycopg2
from datetime import datetime


# Connect to PostgreSQL
conn = psycopg2.connect(
    host="localhost",
    database="system_monitoring",
    user="your_user",
    password="your_password"
)
cursor = conn.cursor()

def log_metrics():
    for proc in psutil.process_iter(['pid', 'name', 'memory_info', 'cpu_times', 'create_time', 'status']):
        try:
            p = psutil.Process(proc.info['pid'])
            mem = p.memory_info()
            cpu = p.cpu_times()
            affinity = ",".join(map(str, p.cpu_affinity())) if hasattr(p, "cpu_affinity") else ""

            data = (
                datetime.now().isoformat(),
                proc.info['name'],
                proc.info['pid'],
                mem.rss,
                mem.vms,
                getattr(mem, "private", None),
                mem.rss,
                mem.vms,
                getattr(mem, "paged_pool", None),
                mem.rss,
                mem.vms,
                cpu.user,
                cpu.system,
                cpu.user + cpu.system,
                affinity,
                datetime.fromtimestamp(p.create_time()).isoformat(),
                not p.is_running() or p.status() == psutil.STATUS_ZOMBIE,
                p.status() != psutil.STATUS_STOPPED
            )

            cursor.execute('''
                INSERT INTO process_metrics VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            ''', data)

        except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
            continue

    conn.commit()

log_metrics()
conn.close()


SQL Server

In [ ]:
CREATE TABLE process_metrics (
    timestamp NVARCHAR(50),
    name NVARCHAR(255),
    pid INT,
    ws BIGINT,
    vm BIGINT,
    private_memory BIGINT,
    working_set BIGINT,
    virtual_memory BIGINT,
    paged_memory BIGINT,
    peak_working_set BIGINT,
    peak_virtual_memory BIGINT,
    cpu_user FLOAT,
    cpu_system FLOAT,
    cpu_total FLOAT,
    processor_affinity NVARCHAR(MAX),
    start_time NVARCHAR(50),
    has_exited BIT,
    responding BIT
);


In [ ]:
import psutil
import pyodbc
from datetime import datetime

# SQL Server connection string
conn = pyodbc.connect(
    r'DRIVER={ODBC Driver 17 for SQL Server};'
    r'SERVER=localhost;'
    r'DATABASE=SystemMonitoring;'
    r'UID=your_user;'
    r'PWD=your_password'
)
cursor = conn.cursor()

def log_metrics():
    for proc in psutil.process_iter(['pid', 'name', 'memory_info', 'cpu_times', 'create_time', 'status']):
        try:
            p = psutil.Process(proc.info['pid'])
            mem = p.memory_info()
            cpu = p.cpu_times()
            affinity = ",".join(map(str, p.cpu_affinity())) if hasattr(p, "cpu_affinity") else ""

            data = (
                datetime.now().isoformat(),
                proc.info['name'],
                proc.info['pid'],
                mem.rss,
                mem.vms,
                getattr(mem, "private", None),
                mem.rss,
                mem.vms,
                getattr(mem, "paged_pool", None),
                mem.rss,
                mem.vms,
                cpu.user,
                cpu.system,
                cpu.user + cpu.system,
                affinity,
                datetime.fromtimestamp(p.create_time()).isoformat(),
                int(not p.is_running() or p.status() == psutil.STATUS_ZOMBIE),
                int(p.status() != psutil.STATUS_STOPPED)
            )

            cursor.execute('''
                INSERT INTO process_metrics VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', data)

        except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
            continue

    conn.commit()

log_metrics()
conn.close()
